In [ ]:
! pip install transformers
! pip install datasets
! pip install accelerate -U
! pip install sentencepiece
! pip install rouge_score
! pip install transformers[torch]
! pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [ ]:
import torch
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import (
    PegasusForConditionalGeneration,
    PegasusTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    logging
)

import nltk
from tabulate import tabulate

In [ ]:
# Set seed for reproducibility
torch.manual_seed(1234)

In [ ]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv("news_articles.csv", dtype=pd.StringDtype(), usecols=range(3))
df = df.drop(columns=['category'])

In [ ]:
print(len(df))

6877


In [ ]:
def clean_data(df):
    # Remove rows with missing headlines/body
    df = df.dropna(axis=0).copy()

    # Convert to lowercase
    df['body'] = df['body'].apply(lambda row: row.lower())
    df['title'] = df['title'].apply(lambda row: row.lower())

    # Remove headline from body
    df['body'] = df.apply(
        lambda row: row['body'].replace(row['title'], ''), axis=1)

    return df

df = clean_data(df)
dataset = Dataset.from_pandas(df)
train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

In [ ]:
# dataset = load_dataset("csv", data_files="news_articles.csv")
# dataset = dataset["train"].train_test_split(test_size=0.1)

In [ ]:
# for cat in ['train', 'test']:
#     column_names = dataset[cat].column_names
#     def filter_missing(example):
#         for column in column_names:
#             if not example[column]:
#                 return False
#         return True
#     dataset[cat] = dataset[cat].filter(filter_missing)

In [ ]:
ENCODER_MAX_LENGTH = 512
DECODER_MAX_LENGTH = 64

def batch_tokenize_preprocess(batch, tokenizer):
    source = [str(item) for item in batch["body"]]
    target = [str(item) for item in batch["title"]]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=ENCODER_MAX_LENGTH)
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=DECODER_MAX_LENGTH)

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(batch, tokenizer),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(batch, tokenizer),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/6184 [00:00<?, ? examples/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

In [ ]:
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge", trust_remote_code=True)


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-11-0b1b6af779a9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge", trust_remote_code=True)


In [ ]:
# max_input_length = 1024
# max_target_length = 64

# def tokenize_function(item, prev):
#     model_inputs = tokenizer(item["body"], max_length=max_input_length, truncation=True)
#     labels = tokenizer(text_target=item["title"], max_length=max_target_length, truncation=True)
#     if not prev:
#       model_inputs["labels"] = [labels["input_ids"]]
#       model_inputs["input_ids"] = [model_inputs["input_ids"]]
#       model_inputs["attention_mask"] = [model_inputs["attention_mask"]]
#     else:
#       prev['input_ids'].append(model_inputs['input_ids'])
#       prev['attention_mask'].append(model_inputs['attention_mask'])
#       prev['labels'].append(labels["input_ids"])
#       model_inputs = prev
#     return model_inputs

In [ ]:
# max_input_length = 512
# max_target_length = 32

# def tokenize_function(item):
#     # print(item["body"][0])
#     # print(item["title"][0])
#     inputs = [doc for doc in item["body"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
#     labels = tokenizer(text_target=item["title"], max_length=max_target_length, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [ ]:
# for idx, sentence in enumerate(dataset["train"]["body"]):
#     if not isinstance(sentence, str):
#         print(sentence, idx)

In [ ]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# print(tokenized_datasets)

In [ ]:
# from evaluate import load
# metric = load("rouge")

In [ ]:
# import nltk
# import numpy as np

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

#     # Note that other metrics may not have a `use_aggregator` parameter
#     # and thus will return a list, computing a metric for each sentence.
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
#     # Extract a few results
#     result = {key: value * 100 for key, value in result.items()}

#     # Add mean generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    save_total_limit=3,
    logging_steps=1000,
    eval_steps=3000,
    save_steps=3000,
    evaluation_strategy='steps',
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True
)

early_stop = EarlyStoppingCallback(3, 0)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stop]
)

In [ ]:
# from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# batch_size = 1
# training_args = Seq2SeqTrainingArguments(
#     output_dir=f"pegasus-finetuned",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True,
# )

In [ ]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     # compute_metrics=compute_metrics
# )

In [ ]:
logging.set_verbosity_error()
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=1546, training_loss=4.046121417165265, metrics={'train_runtime': 2037.9494, 'train_samples_per_second': 3.034, 'train_steps_per_second': 0.759, 'total_flos': 8934223968731136.0, 'train_loss': 4.046121417165265, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.6409006118774414,
 'eval_rouge1': 35.9641,
 'eval_rouge2': 16.514,
 'eval_rougeL': 32.1848,
 'eval_rougeLsum': 32.314,
 'eval_gen_len': 15.5596,
 'eval_runtime': 319.3164,
 'eval_samples_per_second': 2.155,
 'eval_steps_per_second': 0.539,
 'epoch': 1.0}

In [ ]:
# src_text = [

#     """
# For HuffPost’s #LoveTakesAction series, we’re telling stories of how people are standing up to hate and supporting those most threatened. What will you stand up for? Tell us with #LoveTakesAction. Anish Kapoor, Steve McQueen, Laurie Anderson, Ed Ruscha, Matthew Barney, Rosalind Krauss, Hank Willis Thomas, Catherine Opie and Yinka Shonibare are among the artists, writers, musicians and curators from around the world committed to using art as a mode of resistance. Over 200 international creatives have signed on to the global art coalition “Hands Off Our Revolution,” which implores artists of all media to channel the anger, sadness, hope and empathy ignited by present affairs into their work, providing viewers, readers and listeners with models of revolt and visions of a harmonious, peaceful future. The initiative came together in response to recent events ― including Brexit, President Donald Trump’s election and the ongoing refugee crisis. As artist Adam Broomberg explained to The Guardian, such instances of xenophobia and populism are interrelated, and artists must join forces to tackle the larger issue together. “What is important is that it is not just seen as America’s problem, or Europe’s problem,” he said, “so we are planning shows in Mexico and Lagos.”  The coalition pledges to host events and exhibitions around the world bringing contemporary politics into the spotlight, the first of which will be announced in March. This is the latest art world reaction to an uncertain political climate, following a recent artist petition against Trump’s immigration ban. While art certainly possesses the power to amplify marginalized voices and visualize alternate futures, many have argued that protests must extend beyond the insular echo chamber of museums and galleries to truly change minds or make a lasting impact on the lives of others.  “Hands Off” will hopefully find ways to reach past the confines of self-congratulatory, topical exhibitions to benefit those who are in need. So far, the platform seems promising, pledging to donate proceeds from upcoming shows and events to arts and activist causes building coalition. And who better to lead the impending revolution than those guided by creativity, empathy and curiosity?  As put by participating artist Yinka Shonibare: “We must all unite, regardless of race, gender or sexual orientation, to oppose all forms of bigotry. Populism must never be a guide to our conduct, empathy should be our guide. As artists we bear witness and we must never be silent or be silenced.” Read the full manifesto from “Hands Off” below: We are a global coalition affirming the radical nature of art. We believe that art can help counter the rising rhetoric of right-wing populism, fascism and the increasingly stark expressions of xenophobia, racism, sexism, homophobia and unapologetic intolerance. We know that freedom is never granted ― it is won. Justice is never given ― it is exacted. Both must be fought for and protected, yet their promise has seldom been so fragile, so close to slipping from our grasp, as at this moment. As artists, it is our job and our duty to reimagine and reinvent social relations threatened by right-wing populist rule. It is our responsibility to stand together in solidarity. We will not go quietly. It is our role and our opportunity, using our own particular forms, private and public spaces, to engage people in thinking together and debating ideas, with clarity, openness and resilience. “Hands Off” is far from the only creative response to Trump’s rhetoric and policies to emerge this week. The Davis Museum at Wellesley College in Massachusetts, for example, announced that it will be de-installing or shrouding all artwork by immigrants, as well as any art given to the museum by immigrants, to protest Trump’s executive order barring immigrants from seven Muslim-majority nations and blocking Syrian refugees from entering the U.S.  The Museum of the City of New York has also voiced its opinion through art, staging an exhibition called “Muslim in New York” that pays tribute to the legacy of Muslim life throughout the city’s five boroughs. Know a story from your community of people fighting hate and supporting groups who need it? Send news tips to lovetips@huffingtonpost.com.
# """
# ]
# batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
# translated = model.generate(**batch)
# tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
def generate_headline(test_samples, model):
    inputs = tokenizer(
        test_samples["body"],
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [ ]:
# test_samples = validation_data_txt.select(range(16))
# generated_headlines = generate_headline(test_samples, model)[1]
# print(
#     tabulate(
#         zip(
#             range(len(generated_headlines)),
#             generated_headlines,
#             list(test_samples["title"]),
#         ),
#         headers=["Id", "Generated headlines", "Ground Truth"],
#     )
# )

In [ ]:
trainer.save_model("./results/best_pegasus")

In [ ]:
from google.colab import files

FileNotFoundError: Cannot find file: /content/file.zip

In [ ]:
!zip -r /content/file.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/runs/ (stored 0%)
  adding: content/results/runs/May11_21-25-31_aa149d479332/ (stored 0%)
  adding: content/results/runs/May11_21-25-31_aa149d479332/events.out.tfevents.1715465089.aa149d479332.366.1 (deflated 38%)
  adding: content/results/runs/May11_21-25-31_aa149d479332/events.out.tfevents.1715462731.aa149d479332.366.0 (deflated 61%)
  adding: content/results/best_pegasus/ (stored 0%)
  adding: content/results/best_pegasus/special_tokens_map.json (deflated 82%)
  adding: content/results/best_pegasus/training_args.bin (deflated 51%)
  adding: content/results/best_pegasus/model.safetensors (deflated 7%)
  adding: content/results/best_pegasus/config.json (deflated 60%)
  adding: content/results/best_pegasus/spiece.model (deflated 50%)
  adding: content/results/best_pegasus/generation_config.json (deflated 42%)
  adding: content/results/best_pegasus/tokenizer_config.json (deflated 94%)


In [ ]:
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>